## Test di Chow a posteriori

NOTA METODOLOGICA: Questi test vengono eseguiti a **posteriori**, dopo l'identificazione degli outliers attraverso l'analisi dei residui del modello SARIMAX base. I risultati devono essere interpretati con la dovuta cautela metodologica

In [ ]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from scipy.stats import f

# --- CONFIGURAZIONE ---
# Path al file di input (output della Fase 2 - serie stazionarie)
PATH_INPUT_DIR_FASE2 = "/Users/tommaso/Desktop/tesi-inflation-gt/First_Difference_indexes/dati_preparati_fase2"
FILE_SERIE_STAZIONARIE_IN = os.path.join(PATH_INPUT_DIR_FASE2, "indici_gt_nic_stazionari_fase2.csv")

# Path per l'output dei risultati del Test di Chow (modifico il nome per distinguerlo)
PATH_OUTPUT_CHOW_DIR = "/Users/tommaso/Desktop/tesi-inflation-gt/Analisi_Rottura_Chow"
FILE_CHOW_RISULTATI_OUT = os.path.join(PATH_OUTPUT_CHOW_DIR, "risultati_test_chow_outliers_posteriori.csv")

# Nome della colonna dell'inflazione stazionaria (differenziata)
COL_INFLAZIONE_STAZ = 'NIC_destag_ISTAT_diff1'

# Punti di rottura da testare a posteriori (identificati tramite analisi degli outliers)
BREAKPOINT_OUTLIER_GEN_2022 = '2022-01-01'  # Outlier gennaio 2022
BREAKPOINT_OUTLIER_OTT_2022 = '2022-10-01'  # Outlier ottobre 2022

# Numero massimo di lag da considerare per la selezione dell'ordine AR
MAX_AR_LAGS_SELEZIONE = 16

SIGNIFICANCE_LEVEL_CHOW = 0.05

# --- FUNZIONI AUSILIARIE (mantengo le stesse dello script originale) ---
def carica_dati_stazionari(path_file):
    """
    Carico le serie stazionarie (o trasformate) dal CSV salvato dalla Fase 2.
    """
    print(f"--- Caricamento Serie Stazionarie da: {path_file} ---")
    try:
        df = pd.read_csv(path_file, index_col=0)
        df.index = pd.to_datetime(df.index)
        print(f"Serie caricate con successo. Shape: {df.shape}")
        print("--- Fine Caricamento ---\n")
        return df
    except FileNotFoundError:
        print(f"ERRORE: File non trovato: {path_file}")
        return None
    except Exception as e:
        print(f"ERRORE durante il caricamento delle serie stazionarie: {e}")
        return None

def seleziona_ordine_ar_ottimale(y_series, max_lags, criterio='aic'):
    """
    Seleziona l'ordine ottimale per un modello AR usando AIC o BIC.
    Mantengo la stessa logica dello script originale per coerenza metodologica.
    """
    print(f"--- Selezione Ordine AR Ottimale (max_lags={max_lags}, criterio={criterio.upper()}) ---")
    y_cleaned = y_series.dropna()
    if len(y_cleaned) < max_lags + 10:
        print("Dati insufficienti per una selezione affidabile dell'ordine AR. Uso un default di 4 lag.")
        return 4

    selector = ar_select_order(y_cleaned, maxlag=max_lags, trend='c', ic=criterio, old_names=False)
    ordine_ottimale = selector.ar_lags
    
    if isinstance(ordine_ottimale, list) and ordine_ottimale:
        ordine_ottimale = ordine_ottimale[-1]
    elif isinstance(ordine_ottimale, int):
        pass
    else:
        print(f"Attenzione: ar_select_order non ha restituito un ordine chiaro. Uso un default di 4 lag.")
        ordine_ottimale = 4
        
    print(f"Ordine AR ottimale selezionato tramite {criterio.upper()}: {ordine_ottimale}")
    print("--- Fine Selezione Ordine AR ---\n")
    return ordine_ottimale

def chow_test(y_series, X_series, breakpoint_date_str, num_params_k, label_test=""):
    """
    Esegue il test di Chow per un singolo punto di rottura.
    Aggiungo un parametro label_test per identificare quale test stiamo eseguendo.
    """
    print(f"--- Test di Chow per Rottura Strutturale al: {breakpoint_date_str} {label_test} ---")
    
    data_full = pd.concat([y_series, X_series], axis=1).dropna()
    y_full = data_full.iloc[:, 0]
    X_full = data_full.iloc[:, 1:]
    
    n_full = len(y_full)
    if n_full < 2 * num_params_k + 2:
        print("Dati insufficienti per eseguire il test di Chow.")
        return None, None, None, None, None

    breakpoint_ts = pd.Timestamp(breakpoint_date_str)
    if breakpoint_ts <= y_full.index.min() or breakpoint_ts >= y_full.index.max():
        print(f"Punto di rottura {breakpoint_date_str} fuori dal range dei dati. Test non eseguibile.")
        return None, None, None, None, None

    y1 = y_full[y_full.index < breakpoint_ts]
    X1 = X_full[X_full.index < breakpoint_ts]
    y2 = y_full[y_full.index >= breakpoint_ts]
    X2 = X_full[X_full.index >= breakpoint_ts]

    n1, n2 = len(y1), len(y2)
    if n1 < num_params_k + 1 or n2 < num_params_k + 1:
        print(f"Sottocampioni troppo piccoli (n1={n1}, n2={n2}, k={num_params_k}). Test non eseguibile.")
        return None, None, None, None, None
        
    try:
        model_R = sm.OLS(y_full, X_full).fit()
        rss_R = model_R.ssr
        
        model_1 = sm.OLS(y1, X1).fit()
        rss_1 = model_1.ssr
        
        model_2 = sm.OLS(y2, X2).fit()
        rss_2 = model_2.ssr
        
        numerator = (rss_R - (rss_1 + rss_2)) / num_params_k
        denominator = (rss_1 + rss_2) / (n1 + n2 - 2 * num_params_k)
        
        if denominator <= 1e-9:
            print("Errore: Denominatore della statistica F nullo o troppo piccolo. Impossibile calcolare F.")
            return None, None, rss_R, rss_1, rss_2
            
        chow_f_statistic = numerator / denominator
        df_num = num_params_k
        df_den = n1 + n2 - 2 * num_params_k
        p_value = 1 - f.cdf(chow_f_statistic, df_num, df_den)
        
        print(f"RSS Modello Ristretto (intero campione): {rss_R:.4f}")
        print(f"RSS Sottocampione 1 (fino a {breakpoint_date_str}): {rss_1:.4f} (n1={n1})")
        print(f"RSS Sottocampione 2 (da {breakpoint_date_str} in poi): {rss_2:.4f} (n2={n2})")
        print(f"Statistica F di Chow: {chow_f_statistic:.4f}")
        print(f"Gradi di libertà: ({df_num}, {df_den})")
        print(f"P-value: {p_value:.4f}")
        
        if p_value <= SIGNIFICANCE_LEVEL_CHOW:
            print(f"RISULTATO: Si rifiuta H0. Evidenza di rottura strutturale al {SIGNIFICANCE_LEVEL_CHOW*100}% di significatività.")
        else:
            print(f"RISULTATO: Non si rifiuta H0. Nessuna evidenza significativa di rottura strutturale.")
        
        print("--- Fine Test di Chow ---\n")
        return chow_f_statistic, p_value, rss_R, rss_1, rss_2

    except Exception as e:
        print(f"ERRORE durante l'esecuzione del test di Chow: {e}")
        return None, None, None, None, None

# --- ESECUZIONE SCRIPT VERIFICA A POSTERIORI OUTLIERS ---
if __name__ == "__main__":
    print(">>> INIZIO VERIFICA A POSTERIORI: Test di Chow per Outliers Gennaio e Ottobre 2022 <<<\n")
    
    # Aggiungo una nota metodologica importante
    print("NOTA METODOLOGICA: Questi test vengono eseguiti a posteriori, dopo l'identificazione")
    print("degli outliers attraverso l'analisi dei residui del modello SARIMAX base.")
    print("I risultati devono essere interpretati con la dovuta cautela metodologica.\n")

    # Creo la directory di output se non esiste
    os.makedirs(PATH_OUTPUT_CHOW_DIR, exist_ok=True)
    print(f"Directory di output: '{PATH_OUTPUT_CHOW_DIR}'\n")

    # 1. Carico le serie stazionarie dalla Fase 2
    df_serie_stazionarie = carica_dati_stazionari(FILE_SERIE_STAZIONARIE_IN)

    if df_serie_stazionarie is None or COL_INFLAZIONE_STAZ not in df_serie_stazionarie.columns:
        print(f"ERRORE CRITICO: Impossibile caricare il file o colonna '{COL_INFLAZIONE_STAZ}' mancante.")
        exit()

    serie_inflazione = df_serie_stazionarie[COL_INFLAZIONE_STAZ].dropna()

    if serie_inflazione.empty:
        print("ERRORE: Serie dell'inflazione vuota dopo dropna().")
        exit()

    # 2. Seleziono l'ordine AR ottimale (mantengo la stessa metodologia per coerenza)
    ar_lags_ottimali = seleziona_ordine_ar_ottimale(serie_inflazione, 
                                                  max_lags=MAX_AR_LAGS_SELEZIONE, 
                                                  criterio='aic')
    
    if ar_lags_ottimali == 0:
        print("Attenzione: Ordine AR ottimale è 0. Imposto a 1 per procedere.")
        ar_lags_ottimali = 1

    # 3. Preparo i dati per il modello AR(p)
    df_chow_data = pd.DataFrame()
    df_chow_data['y'] = serie_inflazione
    
    for i in range(1, ar_lags_ottimali + 1):
        df_chow_data[f'y_lag_{i}'] = df_chow_data['y'].shift(i)
    
    df_chow_data = sm.add_constant(df_chow_data, prepend=True, has_constant='skip')
    df_chow_data_cleaned = df_chow_data.dropna()
    
    y_chow = df_chow_data_cleaned['y']
    X_chow = df_chow_data_cleaned.drop(columns=['y'])
    num_parametri_modello = X_chow.shape[1]

    print(f"Modello per Test di Chow: {COL_INFLAZIONE_STAZ} ~ const + {ar_lags_ottimali} lag di se stessa.")
    print(f"Numero di parametri (k): {num_parametri_modello}\n")

    if y_chow.empty or X_chow.empty or len(y_chow) < num_parametri_modello * 2 + 2:
        print("ERRORE: Dati insufficienti per i test di Chow.")
        exit()
        
    risultati_chow_lista = []

    # 4. Test di Chow per Gennaio 2022 (primo outlier identificato)
    print("*** Test per Outlier Gennaio 2022 ***")
    f_stat_gen, p_val_gen, rss_r_gen, rss_1_gen, rss_2_gen = chow_test(
        y_chow, X_chow, BREAKPOINT_OUTLIER_GEN_2022, num_parametri_modello, 
        label_test="(Outlier Gennaio 2022)"
    )
    
    if f_stat_gen is not None:
        risultati_chow_lista.append({
            'Breakpoint_Label': 'Outlier_Gennaio_2022',
            'Breakpoint_Date': BREAKPOINT_OUTLIER_GEN_2022,
            'F_Statistic': f_stat_gen,
            'P_Value': p_val_gen,
            'RSS_Ristretto': rss_r_gen,
            'RSS_Sottocampione1': rss_1_gen,
            'RSS_Sottocampione2': rss_2_gen,
            'AR_Lags_Usati': ar_lags_ottimali,
            'Significativo_5pct': 'Sì' if p_val_gen <= SIGNIFICANCE_LEVEL_CHOW else 'No'
        })

    # 5. Test di Chow per Ottobre 2022 (secondo outlier identificato)
    print("*** Test per Outlier Ottobre 2022 ***")
    f_stat_ott, p_val_ott, rss_r_ott, rss_1_ott, rss_2_ott = chow_test(
        y_chow, X_chow, BREAKPOINT_OUTLIER_OTT_2022, num_parametri_modello,
        label_test="(Outlier Ottobre 2022)"
    )
    
    if f_stat_ott is not None:
        risultati_chow_lista.append({
            'Breakpoint_Label': 'Outlier_Ottobre_2022',
            'Breakpoint_Date': BREAKPOINT_OUTLIER_OTT_2022,
            'F_Statistic': f_stat_ott,
            'P_Value': p_val_ott,
            'RSS_Ristretto': rss_r_ott,
            'RSS_Sottocampione1': rss_1_ott,
            'RSS_Sottocampione2': rss_2_ott,
            'AR_Lags_Usati': ar_lags_ottimali,
            'Significativo_5pct': 'Sì' if p_val_ott <= SIGNIFICANCE_LEVEL_CHOW else 'No'
        })

    # 6. Salvo i risultati e genero report finale
    if risultati_chow_lista:
        df_risultati_chow = pd.DataFrame(risultati_chow_lista)
        try:
            df_risultati_chow.to_csv(FILE_CHOW_RISULTATI_OUT, index=False)
            print(f"Risultati salvati in: {FILE_CHOW_RISULTATI_OUT}")
            print("\n=== RIEPILOGO RISULTATI VERIFICA A POSTERIORI ===")
            print(df_risultati_chow[['Breakpoint_Label', 'F_Statistic', 'P_Value', 'Significativo_5pct']])
            
            # Genero interpretazione dei risultati
            print("\n=== INTERPRETAZIONE ===")
            for _, row in df_risultati_chow.iterrows():
                label = row['Breakpoint_Label']
                p_val = row['P_Value']
                significativo = row['Significativo_5pct']
                
                if significativo == 'Sì':
                    print(f"- {label}: ROTTURA STRUTTURALE CONFERMATA (p-value = {p_val:.4f})")
                else:
                    print(f"- {label}: Nessuna evidenza di rottura strutturale (p-value = {p_val:.4f})")
                    
            print("\nNOTA: Questi risultati, ottenuti a posteriori, forniscono verifica formale")
            print("delle rotture strutturali corrispondenti agli outliers identificati empiricamente.")
            
        except Exception as e_save:
            print(f"ERRORE durante il salvataggio: {e_save}")
    else:
        print("Nessun risultato valido da salvare.")

    print("\n>>> VERIFICA A POSTERIORI COMPLETATA <<<")


>>> INIZIO VERIFICA A POSTERIORI: Test di Chow per Outliers Gennaio e Ottobre 2022 <<<

NOTA METODOLOGICA: Questi test vengono eseguiti a posteriori, dopo l'identificazione
degli outliers attraverso l'analisi dei residui del modello SARIMAX base.
I risultati devono essere interpretati con la dovuta cautela metodologica.

Directory di output: '/Users/tommaso/Desktop/tesi-inflation-gt/Analisi_Rottura_Chow'

--- Caricamento Serie Stazionarie da: /Users/tommaso/Desktop/tesi-inflation-gt/First_Difference_indexes/dati_preparati_fase2/indici_gt_nic_stazionari_fase2.csv ---
Serie caricate con successo. Shape: (252, 3)
--- Fine Caricamento ---

--- Selezione Ordine AR Ottimale (max_lags=16, criterio=AIC) ---
Ordine AR ottimale selezionato tramite AIC: 16
--- Fine Selezione Ordine AR ---

Modello per Test di Chow: NIC_destag_ISTAT_diff1 ~ const + 16 lag di se stessa.
Numero di parametri (k): 17

*** Test per Outlier Gennaio 2022 ***
--- Test di Chow per Rottura Strutturale al: 2022-01-01 (Outlie

/Users/tommaso/my_python_env/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/tommaso/my_python_env/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
